In [8]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col,lag,when
from pyspark.sql.window import Window


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

In [3]:
Transactions = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Transactions") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

Transactions.show()

+----------+----------+--------+------+
|account_id|  dayvalue|    type|amount|
+----------+----------+--------+------+
|         1|2021-11-07| Deposit|  2000|
|         1|2021-11-09|Withdraw|  1000|
|         1|2021-11-11| Deposit|  3000|
|         2|2021-11-07| Deposit|  7000|
|         2|2021-11-12|Withdraw|  7000|
+----------+----------+--------+------+



In [12]:
Transactions\
    .withColumn("prev_val",lag("amount").over(Window.partitionBy(col("account_id")).orderBy(col("dayvalue"))))\
    .withColumn("prev_val",when(col("prev_val").isNull(),0).otherwise(col("prev_val")))\
    .withColumn("result",when(col("type")=='Deposit',col("amount")+col("prev_val")).otherwise(col("prev_val")-col("amount")))\
    .select("account_id","dayvalue","result")\
    .show()

+----------+----------+------+
|account_id|  dayvalue|result|
+----------+----------+------+
|         1|2021-11-07|  2000|
|         1|2021-11-09|  1000|
|         1|2021-11-11|  4000|
|         2|2021-11-07|  7000|
|         2|2021-11-12|     0|
+----------+----------+------+

